In [283]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df = pd.read_excel('../csv&excel/영화데이터.xlsx')
kosdaq = pd.read_csv('../csv&excel/kosdaq.csv')

In [284]:
df.drop(columns=['순번', '감독', '제작사', '수입사', '영화유형', '영화형태'], axis=1, inplace=True)

In [285]:
## 평균제작비 기준으로 데이터 정제
# 연도만 추출하여 새로운 파생변수 개봉연도 생성
df['개봉일'] = pd.to_datetime(df['개봉일'])
df['개봉연도'] = df['개봉일'].dt.year

# 새로운 파생변수 평균 제작비 컬럼 생성
def change(x):
    if x == 2009:
        result = 23.1
    elif x == 2010:
        result = 21.6
    elif x == 2011:
        result = 22.7
    elif x == 2012:
        result = 20.3
    elif x == 2013:
        result = 21.4
    elif x == 2014:
        result = 20.1
    elif x == 2015:
        result = 19.9
    elif x == 2016:
        result = 24.0
    elif x == 2017:
        result = 26.3
    elif x == 2018:
        result = 26.8
    elif x == 2019:
        result = 29.0
    else:
        result = '!'
    return result

df['평균 제작비'] = df['개봉연도'].apply(change)
# 매출액 억단위로 변경
df['수정전국매출액']  =  df['전국매출액'] / 100000000
# 평균 제작비보다 매출액이 더 높은 데이터만 추출
df = df[df['수정전국매출액'] >= df['평균 제작비']]
df.reset_index(drop='index', inplace=True)

In [286]:
# 영화명 중 중복 데이터 확인 및 네이밍 작업
df.loc[146, '영화명'] = '스파이(한)'
df.loc[248, '영화명'] = '스파이(미)'
df.loc[83, '영화명'] = '미녀와 야수(미)'
df.loc[887, '영화명'] = '미녀와 야수(프)'
df.loc[321, '영화명'] = '나를 찾아줘(미)'
df.loc[625, '영화명'] = '나를 찾아줘(한)'

### 국적 및 영화등급컬럼 더미화

In [287]:
## 국적 컬럼 변경 (국내 : 1, 해외 : 0)
nation = {
    "한국": 1,
}

df['국적'] = df['국적'].map(nation).fillna(0)

In [288]:
## 영화등급 컬럼 변경
grade = {
    "청소년관람불가": 0,
    "15세이상관람가": 1,
    "12세이상관람가": 2,
    "전체관람가": 3,
    "청소년관람불가,15세이상관람가" : 0,
}

df['등급'] = df['등급'].map(grade)

### 배급사 컬럼의 세분화->3개의 컬럼 생성

In [289]:
#국내배급사 컬럼 생성
# 배급사 컬럼만 추출
gg = df['배급사']

# 배급사 컬럼에서 외국 영화사명 포함된 values 추출 및 병합
a1 = gg[gg.str.contains("디즈니")]
a2 = gg[gg.str.contains("워너브러더스")]
a3 = gg[gg.str.contains("폭스")]
a4 = gg[gg.str.contains("소니")]
a5 = gg[gg.str.contains("유니버설")]
kkk = pd.concat([a1, a2, a3, a4, a5], axis=0)

for i in kkk:
    for j in range(len(df)):
        if i in df['배급사'][j]:
            df.loc[j, '국내배급사'] = 1

df['국내배급사'] = df['국내배급사']
df['국내배급사'].fillna(0, inplace=True)

In [290]:
# 상장배급사 컬럼생성
# 배급사 컬럼에서 상장된 회사 추출 및 병합
b1 = gg[gg.str.contains("씨제이")]
b2 = gg[gg.str.contains("CJ")]
b3 = gg[gg.str.contains("CGV")]
b4 = gg[gg.str.contains("롯데")]
b5 = gg[gg.str.contains("NEW")]
b6 = gg[gg.str.contains("쇼박스")]
b7 = gg[gg.str.contains("스튜디오미르")]
b8 = gg[gg.str.contains("위지윅스튜디오")]
b9 = gg[gg.str.contains("덱스터")]
b10 = gg[gg.str.contains("samg엔터")]
b11= gg[gg.str.contains("애니플러스")]
b12 = gg[gg.str.contains("바른손이앤에이")]
b13 = gg[gg.str.contains("판타지오")]
b14= gg[gg.str.contains("스튜디오산타클로스")]
b15= gg[gg.str.contains("아센디오")]
b16 = gg[gg.str.contains("IHQ")]
b17= gg[gg.str.contains("콘텐트리중앙")]

kkkk = pd.concat([b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b12, b13, b14, b15, b16, b17], axis=0)

# 새로 생성된 컬럼 범주화(0,1)

for i in kkkk:
    for j in range(len(df)):
        if i in df['배급사'][j]:
            df.loc[j, '상장배급사'] = 1

df['상장배급사'].fillna(0, inplace=True)

In [291]:
# 주요배급사 컬럼생성
# 배급사(메이저 비메이저) 컬럼 0,1로 변환(바이너리)후 새 컬럼에 삽입
df['배급사'] = df['배급사'].fillna('aa')

ls = ['이엔엠', 'CJ', '롯데', '쇼박스', '판다', '메가박스', '워너', '유니버설', '소니픽', '디즈니', '판씨', '키다리', 'NEW', '엣나인', \
      '이수', '리틀', '스마일이', '팝엔', '오퍼스', '코리아스', '이십세기', '올스타']

for i in ls:
    for j in range(len(df)):
        if i in df['배급사'][j]:
            df.loc[:, '배급사'][j] = 'O'
            
df['주요배급사'] = df['배급사'].map(lambda x: 1 if x == 'O' else 0)

df['주요배급사'].value_counts()

# 배급사 컬럼 삭제
df = df.drop('배급사', axis=1)

In [292]:
## 해당일관객수 데이터 생성
import pandas as pd
# 경고 메세지 삭제
import warnings
warnings.filterwarnings('ignore')

# 한글 깨짐 방지
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
temp = pd.read_csv('../csv&excel/day_data2.csv', index_col=0)
# 개봉일 2010-01-01 이후 영화만 추출
temp = temp[temp['개봉일'] >= '2010-01-01']
temp.reset_index(drop='index', inplace=True)
# 영화명 앞 뒤 공백 제거
temp["영화명"] = temp["영화명"].apply(lambda x : x.strip())
# 중복 영화 변경(국적으로 나누기)
temp.loc[12537, '영화명'] = '스파이(한)'
temp.loc[18491, '영화명'] = '스파이(미)'
temp.loc[15283, '영화명'] = '미녀와 야수(프)'
temp.loc[24732, '영화명'] = '미녀와 야수(미)'
temp.loc[16517, '영화명'] = '나를 찾아줘(미)'
temp.loc[34236, '영화명'] = '나를 찾아줘(한)'
temp['해당일'] = pd.to_datetime(temp['해당일'])
temp['개봉일'] = pd.to_datetime(temp['개봉일'])
# 필요 없는 컬럼 삭제
temp_1 = temp.drop(['전일 대비 관객수 증감분', '전일 대비 매출액 증감분', '전일 대비 매출액 증감 비율', '누적매출액', '스크린수', '상영된 횟수', '순위', '전일대비 순위의 증감', '신규진입여부' ], axis=1)
# 해당일과 개봉일이 같은 row만 선별
temp2 = temp_1[temp_1['해당일'] == temp['개봉일'] ]
temp2 = temp2.drop(columns=['대표코드', '전일 대비 관객수 증감 비율', '누적관객수', '해당일', '개봉일2', '해당일2'], axis=1).reset_index(drop=True)
temp2.to_csv('../csv&excel/추가변수.csv',index=0)

In [293]:
## 개봉일 데이터 로드
opendf = pd.read_csv('../csv&excel/추가변수.csv')
# 영화명 좌우 빈칸 삭제
df['영화명'] = df['영화명'].str.strip()
opendf['영화명'] = opendf['영화명'].str.strip()
df['영화명'].fillna('1987', inplace=True)
df['영화명'] = df['영화명'].astype(object)

In [294]:
# opendf 영화명 수정
opendf.loc[507, '영화명'] = '나의 PS 파트너'
opendf.loc[627, '영화명'] = '에픽 : 숲속의 전설'
opendf.loc[380, '영화명'] = '존 카터: 바숨 전쟁의 서막'
opendf.loc[346, '영화명'] = '프렌즈: 몬스터섬의비밀'
opendf.loc[504, '영화명'] = '음치 클리닉'
# 로우 데이터와 결합
law_data = pd.merge(df, opendf, how='left', on='영화명')
law_data[law_data['해당일의 관객수'].isna()]

,영화명,개봉일_x,국적,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르,등급,...,개봉연도,평균 제작비,수정전국매출액,국내배급사,상장배급사,주요배급사,해당일의 관객수,개봉일_y,당일 매출액,해당일자 매출총액 대비 해당 영화의 매출비율
489,슈퍼 배드,2010-09-16,0.0,321,9873975412,1043904,2.753597e+09,272176,애니메이션,3,...,2010,21.6,98.739754,1.0,0.0,1,NaN,NaN,NaN,NaN
790,"울지마, 톤즈",2010-09-09,1.0,22,2529614776,441707,1.084744e+09,174478,다큐멘터리,3,...,2010,21.6,25.296148,0.0,0.0,0,NaN,NaN,NaN,NaN
843,내 이름은 칸,2011-03-24,0.0,84,2820922500,381243,1.210674e+09,157628,드라마,2,...,2011,22.7,28.209225,0.0,0.0,0,NaN,NaN,NaN,NaN
862,극장판 짱구는 못말려: 아뵤! 쿵후 보이즈 ~라면 대란~,2018-12-19,0.0,489,2735399650,353012,3.836368e+08,47840,애니메이션,3,...,2018,26.8,27.353997,0.0,1.0,1,NaN,NaN,NaN,NaN


In [295]:
# new_df에는 있지만 opendf(개봉일데이터)에는 없는 영화 4개 드랍 (슈퍼 배드, 울지마 톤즈, 내 이름은 칸, 극장판 짱구는 못말려: 아뵤! 쿵후 보이즈 ~라면 대란~)
law_data = law_data.drop([489, 790, 843, 862], axis = 0)
law_data = law_data[law_data['개봉일_x'] == law_data['개봉일_y']]
# 중복영화 삭제
law_data = law_data.drop_duplicates(subset=['영화명'],keep='first')
law_data.reset_index(drop='index', inplace=True)
law_data.columns

Index(['영화명', '개봉일_x', '국적', '전국스크린수', '전국매출액', '전국관객수', '서울매출액', '서울관객수',
       '장르', '등급', '영화구분', '개봉연도', '평균 제작비', '수정전국매출액', '국내배급사', '상장배급사',
       '주요배급사', '해당일의 관객수', '개봉일_y', '당일 매출액', '해당일자 매출총액 대비 해당 영화의 매출비율'],
      dtype='object')

In [296]:
law_data = law_data[['영화명', '개봉일_x', '국적', '전국스크린수', '전국매출액', '전국관객수', '서울매출액', '서울관객수',
       '장르', '등급', '개봉연도', '평균 제작비', '영화구분', '수정전국매출액', '국내배급사', '상장배급사', '주요배급사', '해당일의 관객수', '당일 매출액', '해당일자 매출총액 대비 해당 영화의 매출비율']]
law_data.columns = ['영화명', '개봉일', '국적', '전국스크린수', '전국매출액', '전국관객수', '서울매출액', '서울관객수',
       '장르', '등급', '개봉연도', '평균 제작비', '영화구분', '수정전국매출액', '국내배급사', '상장배급사', '주요배급사', '개봉일관객수', '개봉일매출액', '개봉일매출비율']
law_data.to_csv('../csv&excel/전처리.csv', index=0)

### 배우 및 감독가중치 컬럼 생성

In [297]:
import pandas as pd
# 경고 메세지 삭제
import warnings
warnings.filterwarnings('ignore')

# 한글 깨짐 방지
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
total = pd.read_csv('../csv&excel/전처리.csv')
actor = pd.read_csv('../csv&excel/배우_1.csv',index_col=0)
director = pd.read_csv('../csv&excel/director.csv', index_col=0)

In [298]:
df = pd.read_csv('../csv&excel/배우_1.csv', index_col=0)
# 0~2 컬럼 value_counts 값
a = df['0'].value_counts()
b = df['1'].value_counts()
c = df['2'].value_counts()
# 데이터프레임화
a = pd.DataFrame(a)
b = pd.DataFrame(b)
c = pd.DataFrame(c)
abc = pd.concat([a,b,c], axis=1)
# 영화별 배우 출현 횟수 총 합
aaa = abc.T.sum().sort_values(ascending=False)
# 데이터프레임화
ddd = pd.DataFrame(aaa)

actor_top3 = actor.iloc[: ,[0, 1, 2]]
actor_top3['합산'] = 0
for i in range(1242):
    for j in range(914):
        if actor_top3['0'].values[j] == ddd.index[i]:
           actor_top3['0'].values[j] = int(ddd.values[i])

for i in range(1242):
    for j in range(914):
        if actor_top3['1'].values[j] == ddd.index[i]:
           actor_top3['1'].values[j] = int(ddd.values[i])
        
for i in range(1242):
    for j in range(914):
        if actor_top3['2'].values[j] == ddd.index[i]:
           actor_top3['2'].values[j] = int(ddd.values[i])
# 배우 출연 횟수 환인
actor_top3.T.sum()

0      39
1      21
2      27
3      29
4      22
       ..
909    10
910    19
911     1
912     7
913     6
Length: 914, dtype: object

In [299]:
## 원본데이터 출연횟수 및 배우가중치 컬럼 생성
total['출연횟수'] = actor_top3.T.sum()
ls = []

for i in range(len(total)):
    if total['전국관객수'][i] >= 10000000:
        ls.append(1)
    elif (total['전국관객수'][i] < 10000000) and (total['전국관객수'][i] >= 5000000):
        ls.append(10)
    elif (total['전국관객수'][i] < 5000000) and (total['전국관객수'][i] >= 1000000):
        ls.append(20)
    else:
        ls.append(30)
    
total['영화순위가중치'] = ls
total['배우가중치'] = total['출연횟수'] / total['영화순위가중치']

In [300]:
## 감독횟수 및 감독가중치 컬럼 생성
ls_d = []

ls_d = director['0'].value_counts()
for i in range(579):
    for j in range(914):
        if director['0'].values[j] == ls_d.index[i]:
           director['0'].values[j] = int(ls_d.values[i])

director.drop(columns='1', inplace=True)
total['감독횟수'] = director['0']
total['감독가중치'] = total['감독횟수'] / total['영화순위가중치']

In [301]:
total.columns

Index(['영화명', '개봉일', '국적', '전국스크린수', '전국매출액', '전국관객수', '서울매출액', '서울관객수', '장르',
       '등급', '개봉연도', '평균 제작비', '영화구분', '수정전국매출액', '국내배급사', '상장배급사', '주요배급사',
       '개봉일관객수', '개봉일매출액', '개봉일매출비율', '출연횟수', '영화순위가중치', '배우가중치', '감독횟수',
       '감독가중치'],
      dtype='object')

In [302]:
total = total[['영화명', '개봉일', '국적', '전국스크린수', '전국매출액', '전국관객수', '서울매출액', '서울관객수', '장르',
       '등급', '개봉연도', '평균 제작비', '영화구분', '수정전국매출액', '국내배급사', '상장배급사', '주요배급사', '개봉일관객수',
       '개봉일매출액', '개봉일매출비율', '배우가중치','감독가중치']]
total

,영화명,개봉일,국적,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르,등급,...,영화구분,수정전국매출액,국내배급사,상장배급사,주요배급사,개봉일관객수,개봉일매출액,개봉일매출비율,배우가중치,감독가중치
0,명량,2014-07-30,1.0,1587,135748398910,17613682,3.312123e+10,4163666,사극,1,...,일반영화,1357.483989,0.0,1.0,1,682701.0,4.708879e+09,61.6,39.0,2.0
1,극한직업,2019-01-23,1.0,1978,139647979516,16264944,3.185866e+10,3638287,코미디,1,...,일반영화,1396.479795,0.0,1.0,1,368582.0,3.004763e+09,73.2,21.0,3.0
2,신과함께-죄와 벌,2017-12-20,1.0,1912,115698654137,14410754,2.753083e+10,3346172,판타지,2,...,일반영화,1156.986541,0.0,1.0,1,406365.0,2.984151e+09,63.2,27.0,3.0
3,국제시장,2014-12-17,1.0,966,110828014630,14245998,2.584252e+10,3233946,드라마,2,...,일반영화,1108.280146,0.0,1.0,1,184756.0,1.356870e+09,31.0,29.0,1.0
4,어벤져스: 엔드게임,2019-04-24,0.0,2835,122182694160,13934592,3.357714e+10,3597963,액션,2,...,일반영화,1221.826942,1.0,0.0,1,1338729.0,9.678990e+09,97.1,22.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,스파이 브릿지,2015-11-05,0.0,425,2015360478,260721,7.222464e+08,90018,스릴러,2,...,일반영화,20.153605,1.0,0.0,1,18404.0,1.352783e+08,5.6,0.333333,0.1
910,함정,2015-09-10,1.0,445,2088685200,257716,4.114176e+08,50477,스릴러,0,...,일반영화,20.886852,0.0,0.0,0,32277.0,2.502558e+08,13.1,0.633333,0.033333
911,파이널 데스티네이션5,2011-09-08,0.0,168,2370348500,257193,8.800060e+08,87499,공포(호러),0,...,일반영화,23.703485,1.0,0.0,1,12613.0,1.180325e+08,7.5,0.033333,0.066667
912,언브로큰,2015-01-07,0.0,401,1997947292,256475,6.057764e+08,75489,액션,1,...,일반영화,19.979473,1.0,0.0,1,26707.0,1.993983e+08,5.8,0.233333,0.033333


In [303]:
total.replace([np.nan, np.inf, -np.inf], 0, inplace=True)
total.to_csv('../csv&excel/전처리_2.csv', index=0)